<a href="https://colab.research.google.com/github/archiechang/study/blob/master/Data%20Analytics/Data%20100/41-50/41-50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd